# Training and Deploying a Custom Image Classifier with Amazon SageMaker

## Configure where to fetch our training data

All of our images live inside an S3 bucket, organized into folders in a structure similar to this:

```
my_training_classes
├── person
│   ├── han.jpg
│   ├── leia.jpg
|   ├── luke.jpg
│   └── . . .
└── ship
│   ├── millenium_falcon.jpg
│   ├── tie-fighter.jpg    
│   ├── x-wing.jpg
│   ├── . . .
└── . . .
```

In [1]:
# An S3 Bucket Name
data_bucket_name='vision-recycle'

# A prefix name inside the S3 bucket containing sub-folders of images (one per label class)
dataset_name = 'pryan-test' 


## Setting up the environment
Here we set up the linkage and authentication to AWS services

- The role used to give learning and hosting access to your data. This will automatically be obtained from the role used to start the notebook
- A `session` variable that holds some configuration state for interacting with SageMaker from Python and contains some methods for preparing input data
- A reference to the Amazon sagemaker image classification docker image 

More info about the SageMaker built-in Image Classification algorithm here: https://docs.aws.amazon.com/sagemaker/latest/dg/image-classification.html

In [2]:
import sagemaker
from sagemaker import get_execution_role
from sagemaker.amazon.amazon_estimator import get_image_uri

role = get_execution_role()
sess = sagemaker.Session()

training_image = get_image_uri(sess.boto_region_name, 'image-classification', repo_version="latest")

## Preparing data for our model
Before we can train our model, we need to:

- Create some files that will teach SageMaker about the images in each of our classes
- Upload these additional files to S3
- Configure our model to use these files for training and validating

### Find the im2rec.py script on this system
The SageMaker image classifier algorithm needs to know about which images belong to which classes. We provide this data using either LST or RecordIO files. We'll use a python script called `im2rec.py` to create these files.

More info here: https://docs.aws.amazon.com/sagemaker/latest/dg/image-classification.html#IC-inputoutput

In [3]:
# Find im2rec in our environment and set up some other vars in our environemnt

base_dir='/tmp'

%env BASE_DIR=$base_dir
%env S3_DATA_BUCKET_NAME = $data_bucket_name
%env DATASET_NAME = $dataset_name

import sys,os

suffix='/mxnet/tools/im2rec.py'
im2rec = list(filter( (lambda x: os.path.isfile(x + suffix )), sys.path))[0] + suffix
%env IM2REC=$im2rec

env: BASE_DIR=/tmp
env: S3_DATA_BUCKET_NAME=vision-recycle
env: DATASET_NAME=pryan-test
env: IM2REC=/home/ec2-user/anaconda3/envs/mxnet_p36/lib/python3.6/site-packages/mxnet/tools/im2rec.py


### Get our training images from S3
In order to create training and validation RecordIO files, we need to download our images to our local filesystem.

In [6]:
# Pull our images from S3
!aws s3 sync s3://$S3_DATA_BUCKET_NAME/$DATASET_NAME $BASE_DIR/$DATASET_NAME --quiet

### Create RecordIO files from our training images
The `im2rec.py` script can create LST files and/or RecordIO files from our training data. 

More info here: https://mxnet.incubator.apache.org/versions/master/faq/recordio.html

In [7]:
%%bash
# Use the IM2REC script to convert our images into RecordIO files

# Clean up our working dir of existing LST and REC files
cd $BASE_DIR
rm *.rec
rm *.lst

# First we need to create two LST files (training and test lists), noting the correct label class for each image
# We'll also save the output of the LST files command, since it includes a list of all of our label classes
echo "Creating LST files"
python $IM2REC --list --recursive --pass-through --test-ratio=0.3 --train-ratio=0.7 $DATASET_NAME $DATASET_NAME > ${DATASET_NAME}_classes

echo "Label classes:"
cat ${DATASET_NAME}_classes

# Then we create RecordIO files from the LST files
echo "Creating RecordIO files"
python $IM2REC --num-thread=4 ${DATASET_NAME}_train.lst $DATASET_NAME
python $IM2REC --num-thread=4 ${DATASET_NAME}_test.lst $DATASET_NAME
ls -lh *.rec

Creating LST files
Label classes:
water_bottle 0
Creating RecordIO files
Creating .rec file from /tmp/pryan-test_train.lst in /tmp
time: 1.101731777191162  count: 0
Creating .rec file from /tmp/pryan-test_test.lst in /tmp
time: 0.006418704986572266  count: 0
-rw-rw-r-- 1 ec2-user ec2-user 160K Oct 30 03:42 pryan-test_test.rec
-rw-rw-r-- 1 ec2-user ec2-user 365K Oct 30 03:42 pryan-test_train.rec


### Upload our training and test data RecordIO files so we can train with them
Now that we have our training and test .rec files, we upload them to S3 so SageMaker can use them for training

In [8]:
# Upload our train and test RecordIO files to S3 in the bucket that our sagemaker session is using
bucket = sess.default_bucket()

s3train_path = 's3://{}/{}/train/'.format(bucket, dataset_name)
s3validation_path = 's3://{}/{}/validation/'.format(bucket, dataset_name)

# Clean up any existing data
!aws s3 rm s3://{bucket}/{dataset_name}/train --recursive
!aws s3 rm s3://{bucket}/{dataset_name}/validation --recursive

# Upload the rec files to the train and validation channels
!aws s3 cp /tmp/{dataset_name}_train.rec $s3train_path
!aws s3 cp /tmp/{dataset_name}_test.rec $s3validation_path

upload: ../../../../tmp/pryan-test_train.rec to s3://sagemaker-us-east-1-023375022819/pryan-test/train/pryan-test_train.rec
upload: ../../../../tmp/pryan-test_test.rec to s3://sagemaker-us-east-1-023375022819/pryan-test/validation/pryan-test_test.rec


### Configure the data for our model training to use
Finally, we tell SageMaker where to find these RecordIO files to use for training

In [9]:
train_data = sagemaker.session.s3_input(
    s3train_path, 
    distribution='FullyReplicated', 
    content_type='application/x-recordio', 
    s3_data_type='S3Prefix'
)

validation_data = sagemaker.session.s3_input(
    s3validation_path, 
    distribution='FullyReplicated', 
    content_type='application/x-recordio', 
    s3_data_type='S3Prefix'
)

data_channels = {'train': train_data, 'validation': validation_data}

## Training
Now it's time to train our model!

### Create an image classifier object with some base configuration
More info here: https://sagemaker.readthedocs.io/en/stable/estimators.html#sagemaker.estimator.Estimator

In [10]:
s3_output_location = 's3://{}/{}/output'.format(bucket, dataset_name)

image_classifier = sagemaker.estimator.Estimator(
    training_image,
    role, 
    train_instance_count=1, 
    train_instance_type='ml.p3.2xlarge',
    output_path=s3_output_location,
    sagemaker_session=sess
)

### Set some training hyperparameters

Finally, before we train, we provide some additional configuration parameters for the training.

More info here: https://docs.aws.amazon.com/sagemaker/latest/dg/IC-Hyperparameter.html

In [11]:
num_classes=! ls -l {base_dir}/{dataset_name} | wc -l
num_classes=int(num_classes[0]) - 1

num_training_samples=! cat {base_dir}/{dataset_name}_train.lst | wc -l
num_training_samples = int(num_training_samples[0])

# Learn more about the Sagemaker built-in Image Classifier hyperparameters here: https://docs.aws.amazon.com/sagemaker/latest/dg/IC-Hyperparameter.html

# These hyperparameters we won't want to change, as they define things like
# the size of the images we'll be sending for input, the number of training classes we have, etc.
base_hyperparameters=dict(
    use_pretrained_model=1,
    image_shape='3,224,224',
    num_classes=num_classes,
    num_training_samples=num_training_samples,
)

# These are hyperparameters we may want to tune, as they can affect the model training success:
hyperparameters={
    **base_hyperparameters, 
    **dict(
        learning_rate=0.001,
        mini_batch_size=5,
    )
}


image_classifier.set_hyperparameters(**hyperparameters)

hyperparameters

{'use_pretrained_model': 1,
 'image_shape': '3,224,224',
 'num_classes': 1,
 'num_training_samples': 21,
 'learning_rate': 0.001,
 'mini_batch_size': 5}

### Start the training
Train our model!

This will take some time because it's provisioning a new container runtime to train our model, then the actual training happens, then the trained model gets uploaded to S3 and the container is shut down.

More info here: https://sagemaker.readthedocs.io/en/stable/estimators.html#sagemaker.estimator.Estimator.fit

In [12]:
%%time

import time
now = str(int(time.time()))
training_job_name = 'IC-' + dataset_name.replace('_', '-') + '-' + now

image_classifier.fit(inputs=data_channels, job_name=training_job_name, logs=True)

job = image_classifier.latest_training_job
model_path = f"{base_dir}/{job.name}"

print(f"\n\n Finished training! The model is available for download at: {image_classifier.output_path}/{job.name}/output/model.tar.gz")

2019-10-30 03:44:13 Starting - Starting the training job...
2019-10-30 03:44:15 Starting - Launching requested ML instances......
2019-10-30 03:45:19 Starting - Preparing the instances for training......
2019-10-30 03:46:27 Downloading - Downloading input data...
2019-10-30 03:46:48 Training - Downloading the training image...
2019-10-30 03:47:37 Training - Training image download completed. Training in progress..Docker entrypoint called with argument(s): train
[10/30/2019 03:47:40 INFO 139802820364096] Reading default configuration from /opt/amazon/lib/python2.7/site-packages/image_classification/default-input.json: {u'beta_1': 0.9, u'gamma': 0.9, u'beta_2': 0.999, u'optimizer': u'sgd', u'use_pretrained_model': 0, u'eps': 1e-08, u'epochs': 30, u'lr_scheduler_factor': 0.1, u'num_layers': 152, u'image_shape': u'3,224,224', u'precision_dtype': u'float32', u'mini_batch_size': 32, u'weight_decay': 0.0001, u'learning_rate': 0.1, u'momentum': 0}
[10/30/2019 03:47:40 INFO 139802820364096] Rea

## Deploy the trained model
Once a model has been trained, we can use the same `image_classifier` object to create a deployed, fully-managed endpoint.}

More info here: https://sagemaker.readthedocs.io/en/stable/estimators.html#sagemaker.estimator.Estimator.deploy

In [13]:
%%time
# Deploying a model to an endpoint takes a few minutes to complete

deployed_endpoint = image_classifier.deploy(
    initial_instance_count = 1,
    instance_type = 'ml.t2.medium'
)

---------------------------------------------------------------------------------------------------------------------------------------------------!CPU times: user 749 ms, sys: 36.9 ms, total: 785 ms
Wall time: 12min 22s


### Clean up

When we're done with the endpoint, we can just delete it and the backing instances will be released.  Run the following cell to delete the endpoint.

In [ ]:
deployed_endpoint.delete_endpoint()

## Calling a deployed endpoint from Python code

If you want to try using a deployed endpoint from Python, here's a function that you can use. It takes in a path to the image you'd like to classify, and a list of all the classes used for training.

In [45]:
import json
import numpy as np
import os

def classify_deployed(file_name, classes):
    payload = None
    with open(file_name, 'rb') as f:
        payload = f.read()
        payload = bytearray(payload)

    deployed_endpoint.content_type = 'application/x-image'
    result = json.loads(deployed_endpoint.predict(payload))
    print(result)
    best_prob_index = np.argmax(result)
    print(best_prob_index)
    return (classes[best_prob_index], result[best_prob_index])



In [46]:
deployed_endpoint

In [47]:
#file_name = "s3://vision-recycle/pryan-test/water_bottle/water_bottle_0KxQl.png"
file_name = 'plastic-empty-water-bottle-500x500.jpg'
classes = ['water-bottle']
classify_deployed(file_name, classes)

1.0
0


TypeError: 'float' object is not subscriptable

In [28]:
classes[0]

'water-bottle'

## (Optional) Perform Hyperparameter Tuning

Often, you might not know which values for hyperparameters like `learning_rate` and `mini_batch_size` will yield acceptible results. Traditionally, this meant manually running many training jobs with different hyperparameter values, looking at each trained model's performance, and then picking a winner. 

This type of manual tuning is _very_ time consuming, so you can automate this process using automatic model tuning with SageMaker. Here's some example code to illustrate how to start one of these jobs using the SageMaker Python SDK.

More info here about automatic model tuning: https://docs.aws.amazon.com/sagemaker/latest/dg/automatic-model-tuning.html

More info about model tuning for the Image Classification algorithm: https://docs.aws.amazon.com/sagemaker/latest/dg/IC-tuning.html

In [ ]:
from sagemaker.tuner import HyperparameterTuner, IntegerParameter, CategoricalParameter, ContinuousParameter
hyperparameter_ranges = {'optimizer': CategoricalParameter(['sgd', 'adam']),
                         'learning_rate': ContinuousParameter(0.0001, 0.1),
                         'mini_batch_size': IntegerParameter(2, 32),
                        }

objective_metric_name = 'validation:accuracy'

tuner = HyperparameterTuner(image_classifier,
                            objective_metric_name,
                            hyperparameter_ranges,
                            max_jobs=50,
                            max_parallel_jobs=3)

tuner.fit(inputs=data_channels, logs=True, include_cls_metadata=False)

## Great resources to continue your Deep Learning journey

[3Blue1Brown’s YouTube series on Neural Networks ~ 60 Minutes](https://www.youtube.com/playlist?list=PLZHQObOWTQDNU6R1_67000Dx_ZCJB-3pi)

[Fast.ai’s Practical Deep Learning for Coders ~ 14 Hours](http://www.fast.ai/)
    
[Amazon's Machine Learning University ~ More than 45 hours of courses, videos, and labs](https://aws.amazon.com/training/learning-paths/machine-learning/)
    
[Neural Networks and Deep Learning, by Michael Neilsen ~ 6 Chapter Book](http://neuralnetworksanddeeplearning.com/)

[Amazon SageMaker - Fully-managed Platform](https://aws.amazon.com/sagemaker/)
    
[@gabehollombe's](https://twitter.com/gabehollombe) deep learning tools and demos
- [Jupyter Notebooks](https://github.com/gabehollombe-aws/jupyter-notebooks)
- [Webcam S3 Uploader Tool](https://github.com/gabehollombe-aws/webcam-s3-uploader)
- [SageMaker Inference Web Tool](https://github.com/gabehollombe-aws/webcam-sagemaker-inference)
